In [1]:
import numpy as np
import matplotlib.pyplot as plt 
from bs4 import BeautifulSoup
import pandas as pd
import json
import os
import requests
import time
from tqdm import tqdm

In [11]:
url = 'https://raw.githubusercontent.com/Oliverkobaek/thesis_2020/master/APOTEK_20200109.csv'
exceldata = pd.read_csv(url)
exceldata = exceldata.dropna(subset=['Adresse', 'PostNr'])
df=exceldata.reset_index(drop=True)
df.head()

,ApotekNr,Navn,TypeID,Adresse,PostNr,Startdato,Slutdato,amt,Kommune,Region
0,359,Albertslund Apotek,6,Bytorvet 5,2620.0,01Jan1966 0:00:00.000,NaN,1084.0,165.0,1084.0
1,346-1,Allerød Apotek,7,Allerød Stationsvej 9,3450.0,NaN,NaN,1084.0,201.0,1084.0
2,082-1,Allinge Apotek,7,Kirkegade 13,3770.0,01May2016 0:00:00.000,NaN,1084.0,400.0,1084.0
3,175,Allinge Apotek,6,Kirkegade 13,3770.0,01-01-1898 00:00,01May2016 0:00:00.000,1084.0,400.0,1084.0
4,243-1,Amager Strand Apotek,7,Strandlodsvej 41,2300.0,21Sep2015 0:00:00.000,NaN,1084.0,101.0,1084.0


In [12]:
import math
# List for storing names of pharmacies missing start date 
mangler_start = []

# Looping through our dataframe to identify 'nan' in start date
for i in range(len(df)):
    # Using try so we skip all the strings in the column * note that the NaN values are float but the dates are str
    try:
        if math.isnan(df['Startdato'][i]):
            mangler_start.append(df['Navn'][i])
    #skipping if value is not a flaot i.e. it is not nan        
    except Exception:
        continue

In [13]:
len(mangler_start)

153

In [14]:
df

,ApotekNr,Navn,TypeID,Adresse,PostNr,Startdato,Slutdato,amt,Kommune,Region
0,359,Albertslund Apotek,6,Bytorvet 5,2620.0,01Jan1966 0:00:00.000,NaN,1084.0,165.0,1084.0
1,346-1,Allerød Apotek,7,Allerød Stationsvej 9,3450.0,NaN,NaN,1084.0,201.0,1084.0
2,082-1,Allinge Apotek,7,Kirkegade 13,3770.0,01May2016 0:00:00.000,NaN,1084.0,400.0,1084.0
3,175,Allinge Apotek,6,Kirkegade 13,3770.0,01-01-1898 00:00,01May2016 0:00:00.000,1084.0,400.0,1084.0
4,243-1,Amager Strand Apotek,7,Strandlodsvej 41,2300.0,21Sep2015 0:00:00.000,NaN,1084.0,101.0,1084.0
5,022-1,Antvorskov Apotek,7,Idagårdsvej 1,4200.0,01Jun2018 0:00:00.000,NaN,1085.0,330.0,1085.0
6,411,Apopro Online Apotek,6,Ventrupvej 27,2670.0,15Jun2017 0:00:00.000,NaN,1085.0,253.0,1085.0
7,090-7,Apotek Borups Alle,7,Borups Alle 116 st. tv.,2000.0,12Nov2018 0:00:00.000,NaN,1084.0,147.0,1084.0
8,090-1,Apotek Fisketorvet,7,"Fisketorvet, Kalvebod Brygge 59",1561.0,01Aug2015 0:00:00.000,NaN,1084.0,101.0,1084.0
9,090-3,Apotek Humletorvet,7,Humletorvet 23,1799.0,01Nov2016 0:00:00.000,NaN,1084.0,101.0,1084.0


In [15]:
# Importing our search results from the datacvr.virk.dk
# The search was a very general search for "apotek*"
url_virk = 'https://raw.githubusercontent.com/Oliverkobaek/thesis_2020/master/CVR_virk.csv'
cvr_virk = pd.read_csv(url_virk, sep=';')
# We note that we have 855 firms in our search result
print(cvr_virk.shape)


# Selecting only rows with 'Hovedbranche = 477300 Apoteker' to remove every irrelevant firm
cvr_virk_ap = cvr_virk[(cvr_virk.Hovedbranche == '477300 Apoteker') | (cvr_virk.Hovedbranche =='523100 Apoteker')]
cvr_virk_a = cvr_virk_ap.reset_index(drop=True)
print(cvr_virk_a.shape)

(855, 12)
(378, 12)


In [16]:
# P-enheder
url_p = 'https://raw.githubusercontent.com/Oliverkobaek/thesis_2020/master/CVR_P-enheder.csv'
cvr_p = pd.read_csv(url_p, sep=';')
# We note that we have 1707 firms in our search result
print(cvr_p.shape)

# Selecting only rows with 'Hovedbranche = 477300 Apoteker' to remove every irrelevant firm
cvr_p_ap = cvr_p[(cvr_p.Hovedbranche == '477300 Apoteker')| (cvr_p.Hovedbranche =='523100 Apoteker')]
cvr_p_a = cvr_p_ap.reset_index(drop=True)
print(cvr_p_a.shape)

(1707, 11)
(1018, 11)


In [17]:
cvr_p_a.head(3)

,P-nummer,Startdato,Ophørsdato,Navn,Adresse,Postnr.,By,Hovedbranche,Telefonnr,Email,Reklamebeskyttet
0,1023758268,18.08.2018,NaN,Skalborg Apotek,Otto Mønsteds Vej 1,9200.0,Aalborg,477300 Apoteker,21608664.0,063fo@apoteket.dk,Ja
1,1025913309,01.09.2020,NaN,Stensballe Apotek,c/o Superbrugsen Stensballe - Sundgårdsvej 1,8700.0,Horsens,477300 Apoteker,76254200.0,NaN,Ja
2,1008657978,01.01.2000,NaN,Hasseris apotek,Hasseris Bymidte 9,9000.0,Aalborg,477300 Apoteker,NaN,NaN,Nej


In [18]:
# Caseless matching
mangler_start_lower = [x.casefold() for x in mangler_start]

In [19]:
case_cvr_p_a = [x.casefold() for x in cvr_p_a.Navn]
case_cvr_virk_a = [x.casefold() for x in cvr_virk_a.Navn]

In [20]:
cvr_p_a['Navn_Lille']=case_cvr_p_a
cvr_virk_a['Navn_lille']=case_cvr_virk_a

In [21]:
cvr_p_a.shape

(1018, 12)

In [22]:
print(len(case_cvr_p_a), len(case_cvr_virk_a))
samlet_opslag = case_cvr_p_a+case_cvr_virk_a
print(len(samlet_opslag))

1018 378
1396


In [23]:
found=[]
still_missing=[]
for missing in mangler_start_lower:
    if missing in samlet_opslag:
        found.append(missing)
    if missing not in samlet_opslag:
        still_missing.append(missing)
        
print('fundet', len(found), 'mangler endnu', len(still_missing))

fundet 87 mangler endnu 66


In [24]:
#import Levenshtein as lev
#pip install fuzzywuzzy
#pip install python-Levenshtein

In [25]:
matching = {}
for i in range(len(still_missing)):
    for name in range(len(samlet_opslag)):
        if still_missing[i] in samlet_opslag[name]:
            matching[still_missing[i]]=samlet_opslag[name]

matching

{'brørup apotek': 'brørup apotek v/kurt jensen høy',
 'espergærde apotek': 'espergærde apotek, espergærde center',
 'folkebo apoteksudsalg': 'folkebo apoteksudsalg v/bente møgelhøj',
 'ishøj apotek': 'ishøj apotek v/lars østergaard christensen',
 'kolind apotek': 'kolind apoteksudsalg v/lægemiddelstyrelsen',
 'køge torvets apotek': 'køge torvets apotek v/peter schjørring-thyssen ',
 'ryomgård apoteksudsalg': 'ryomgård apoteksudsalg v/lægemiddelstyrelsen'}

In [26]:
print('fundet', len(found), 'mangler endnu', len(still_missing))
for x in matching:
    if x in still_missing:
        still_missing.remove(x)
        found.append(x)
    
print('fundet', len(found), 'mangler endnu', len(still_missing))    

fundet 87 mangler endnu 66
fundet 94 mangler endnu 59


In [27]:
still_missing

['apoteksudsalg fra trekroner apopotek',
 'apoteksudsalg vest',
 'apoteksudsalget herningcentret',
 'apoteksudsalget på kingosvej',
 'apoteksudsalget thors bakke',
 'apoteksudsalget vestamager centret',
 'augustenborg apoteksudsalg',
 'aulum apoteksudsalg',
 'bankagervejs apoteksudsalg',
 'bellinge apoteksudsalg',
 'brede apoteksudsalg',
 'erritsø apoteksudsalg',
 'fensmark apoteksudsalg',
 'frederiksberg dalgas boulevard apotek',
 'glarbjerg apoteksudsalg',
 'grantoften apoteksudsalg',
 'greve apotek (gl)',
 'hasseris bymidte apoteksudsalg',
 'hedegårdens apoteksudsalg',
 'herfølge apoteksudsalg',
 'hjortespring-centrets apoteksudsalg',
 'høje gladsaxe torv apoteksudsalg',
 'høje tåstrup apoteksudsalg',
 'hørning apoteksudsalg',
 'hørve apoteksudsalg',
 'jægerspris apoteksudsalg',
 'kirkebjerg torv apoteksudsalg',
 'kolding storcenter apoteksudsalg',
 'korup apoteksudsalg',
 'københavn apoteket ryparken',
 'køgevej apoteksudsalg',
 'langå apoteksudsalg',
 'løgten apoteksudsalg',
 'løv

In [30]:
cvr_p_a['P-nummer']


url_p = []
for i in range(len(cvr_p_a['P-nummer'])):
    url_p.append('https://datacvr.virk.dk/data/visenhed?enhedstype=produktionsenhed&id='+str(cvr_p_a['P-nummer'][i])+'&soeg='+str(cvr_p_a['P-nummer'][i])+'&language=da')

url_v = []
for i in range(len(cvr_virk_a['CVR-nummer'])):
    url_v.append('https://datacvr.virk.dk/data/visenhed?enhedstype=virksomhed&id='+str(cvr_virk_a['CVR-nummer'][i])+'&soeg='+str(cvr_virk_a['CVR-nummer'][i])+'&language=da')
  


In [31]:
cvr_links_final = url_v + url_p

In [51]:
len(url_p[:500])

500

In [54]:
len(cvr_links_final)

1396

In [58]:
cvr_links = cvr_links_final

dataframe_cvr = []

for link in tqdm(cvr_links):
    response = requests.get(link)
    soup = BeautifulSoup(response.content, "html.parser")
    table_div = soup.find_all('div', class_='table')
    #try: 
    for div in table_div:
        table_hist=div.find('div', {"class":'historisk-data'})
        sup_soup = BeautifulSoup(str(table_hist))
        tds = sup_soup.find_all('div', class_="row historisk-row")
    #except AttributeError:
        #continue
        cvr_data = []
        for td in tds[1:]:
            cells = td.find_all('div', {"data-pdf-class":'column4'})
            cvr_data.append([cell.text.strip('\n')[:-1] for cell in cells])
        
    dataframe_cvr.append(pd.DataFrame(cvr_data))
    time.sleep(20)
    
data_cvr = pd.concat(dataframe_cvr)

  0%|                                                                                         | 0/1396 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [59]:
data_cvr.to_excel('CVR_scraped_resultat_p4.xlsx')

In [38]:
Data_cvr_load = pd.read_excel('CVR_scraped_resultat_2.xlsx')

In [42]:
matching_2 = {}
for i in range(len(still_missing)):
    for name in range(len(Data_cvr_load[2])):
        if still_missing[i] in Data_cvr_load[2][name]:
            matching_2[still_missing[i]]=Data_cvr_load[2][name]

matching_2

{}

In [7]:
dataframe_cvr

[Empty DataFrame
 Columns: []
 Index: [], Empty DataFrame
 Columns: []
 Index: []]

In [368]:
response = requests.get(link)
soup = BeautifulSoup(response.content, "html.parser")
table_div = soup.find_all('div', class_='table')
for div in table_div:
    table_hist=div.find('div', {"class":'historisk-data'})
    sup_soup = BeautifulSoup(str(table_hist))
    tds = sup_soup.find_all('div', class_="row historisk-row")

    #tds = table_hist.find_all('div', class_="row historisk-row")

In [371]:
dataframe_cvr

[            0           1                          2
 1  05.02.2009  31.01.2016  Apoteksudsalget i Erritsø]

In [372]:
tds

[<div class="row historisk-row" data-pdf-class="column12">
 <div class="col-md-2" data-pdf-class="column4">
 <strong>
 Fra </strong>
 </div>
 <div class="col-md-2" data-pdf-class="column4">
 <strong>
 Til </strong>
 </div>
 <div class="col-md-8" data-pdf-class="column4">
 </div>
 </div>, <div class="row historisk-row" data-pdf-class="column12">
 <div class="col-md-2" data-pdf-class="column4">
 01.02.2016 </div>
 <div class="col-md-2" data-pdf-class="column4">
 26.05.2018 </div>
 <div class="col-md-8" data-pdf-class="column4">
 Løve Apoteket Lillebælt Løvbjerg </div>
 </div>, <div class="row historisk-row" data-pdf-class="column12">
 <div class="col-md-2" data-pdf-class="column4">
 05.02.2009 </div>
 <div class="col-md-2" data-pdf-class="column4">
 31.01.2016 </div>
 <div class="col-md-8" data-pdf-class="column4">
 Apoteksudsalget i Erritsø </div>
 </div>]

In [366]:
tds = table_hist.find_all('div', class_="row historisk-row")

In [355]:

tds = table_hist.find_all('div', class_="row historisk-row")

In [342]:
for div in table_div:
    table_hist=div.find('div', {"class":'historisk-data'})

In [343]:
table_hist
tds = table_hist.find_all('div', class_="row historisk-row")
cvr_data = []
for td in tds[1:]:
    cells = td.find_all('div', {"data-pdf-class":'column4'})
    #names = td.find_all('div', class_='col-md-8')
    #cells.append(names)
    cvr_data.append([cell.text.strip('\n')[:-1] for cell in cells])

In [344]:
cvr_data

[['01.02.2016', '26.05.2018', 'Løve Apoteket Lillebælt Løvbjerg'],
 ['05.02.2009', '31.01.2016', 'Apoteksudsalget i Erritsø']]